# Options Backtesting System - Quick Start

This notebook provides a quick introduction to the Options Backtesting System.

## What You'll Learn
1. Setting up a simple backtest
2. Running a strategy
3. Analyzing results
4. Generating visualizations

In [ ]:
# Setup - Add parent directory to path
import sys
from pathlib import Path
code_dir = Path('..') / 'code'
sys.path.insert(0, str(code_dir))

# Imports
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from unittest.mock import Mock

print("Imports successful!")

## Step 1: Create Mock Data

For this example, we'll use mock data instead of a real database.

In [ ]:
from backtester.engine.data_stream import DataStream
from backtester.data.market_data import MarketData, OptionChain
from backtester.core.pricing import BlackScholesPricer

def create_mock_data_stream(start_date, end_date, initial_price=450.0):
    """Create a mock data stream for testing."""
    mock_adapter = Mock()
    mock_adapter.is_connected.return_value = True
    
    data_stream = DataStream(mock_adapter, start_date, end_date, 'SPY')
    
    # Generate price data
    dates = pd.bdate_range(start_date, end_date)
    np.random.seed(42)
    returns = np.random.normal(0.0005, 0.015, len(dates))
    prices = initial_price * np.exp(np.cumsum(returns))
    
    price_df = pd.DataFrame({'date': dates, 'close': prices})
    
    def get_market_data(date):
        row = price_df[price_df['date'] == pd.Timestamp(date)]
        if row.empty:
            return None
        close = row['close'].iloc[0]
        return MarketData('SPY', date, close*0.999, close*1.001, close*0.998, close, 50_000_000)
    
    def get_option_chain(date, expiration=None):
        market_data = get_market_data(date)
        if not market_data:
            return None
        if not expiration:
            expiration = date + timedelta(days=30)
        
        pricer = BlackScholesPricer()
        S, T, r, sigma = market_data.close, (expiration - date).days / 365.25, 0.04, 0.20
        
        calls, puts = [], []
        for pct in range(90, 111, 2):
            K = round(S * pct / 100, 2)
            call_price = pricer.price(S, K, T, r, sigma, 'call')
            put_price = pricer.price(S, K, T, r, sigma, 'put')
            call_greeks = pricer.calculate_greeks(S, K, T, r, sigma, 'call')
            put_greeks = pricer.calculate_greeks(S, K, T, r, sigma, 'put')
            
            calls.append({'strike': K, 'mid': call_price, 'bid': call_price*0.98, 
                         'ask': call_price*1.02, 'implied_volatility': sigma,
                         'volume': 1000, 'open_interest': 5000, **call_greeks})
            puts.append({'strike': K, 'mid': put_price, 'bid': put_price*0.98,
                        'ask': put_price*1.02, 'implied_volatility': sigma,
                        'volume': 1000, 'open_interest': 5000, **put_greeks})
        
        return OptionChain('SPY', date, expiration, calls, puts)
    
    data_stream.get_market_data = get_market_data
    data_stream.get_option_chain = get_option_chain
    
    return data_stream

print("Mock data functions created!")

## Step 2: Create a Strategy

We'll use the Short Straddle strategy that sells ATM options when IV is high.

In [ ]:
from backtester.strategies import ShortStraddleHighIVStrategy

strategy = ShortStraddleHighIVStrategy(
    name='Short Straddle Demo',
    initial_capital=100000.0,
    iv_rank_threshold=50,
    profit_target_pct=0.50,
    stop_loss_pct=2.0
)

print(f"Strategy created: {strategy.name}")
print(f"Initial capital: ${strategy.initial_capital:,.2f}")

## Step 3: Run the Backtest

In [ ]:
from backtester.engine.backtest_engine import BacktestEngine
from backtester.engine.execution import ExecutionModel

# Setup
start_date = datetime(2024, 1, 2)
end_date = datetime(2024, 3, 29)

data_stream = create_mock_data_stream(start_date, end_date)
execution = ExecutionModel(commission_per_contract=0.65, slippage_pct=0.01)

# Run backtest
print("Running backtest...")
engine = BacktestEngine(strategy, data_stream, execution, 100000.0)
results = engine.run()
print("Backtest complete!")

## Step 4: Analyze Results

In [ ]:
from backtester.analytics import PerformanceMetrics

# Display key results
print("="*60)
print("BACKTEST RESULTS")
print("="*60)
print(f"Initial Capital:  ${100000.0:,.2f}")
print(f"Final Equity:     ${results['final_equity']:,.2f}")
print(f"Total Return:     {results['total_return']*100:.2f}%")
print(f"Total P&L:        ${results['final_equity'] - 100000.0:,.2f}")
print()
print(f"Total Trades:     {len(results['trade_log'])}")

# Calculate metrics
equity_curve = results['equity_curve']
returns = equity_curve['equity'].pct_change().dropna()

if len(returns) > 1:
    sharpe = PerformanceMetrics.calculate_sharpe_ratio(returns)
    max_dd = PerformanceMetrics.calculate_max_drawdown(equity_curve)
    print(f"Sharpe Ratio:     {sharpe:.2f}" if sharpe else "Sharpe Ratio:     N/A")
    print(f"Max Drawdown:     {max_dd*100:.2f}%")

## Step 5: Visualize Equity Curve

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Plot equity curve
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(equity_curve.index, equity_curve['equity'], linewidth=2, color='blue')
ax.axhline(y=100000, color='gray', linestyle='--', label='Initial Capital')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Equity ($)', fontsize=12)
ax.set_title('Equity Curve', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

## Step 6: Trade Log Analysis

In [ ]:
trade_log = results['trade_log']

if len(trade_log) > 0:
    print("Trade Log Summary:")
    print(trade_log[['entry_date', 'exit_date', 'pnl', 'return_pct']].head(10))
    
    # Plot P&L distribution
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.hist(trade_log['pnl'], bins=20, edgecolor='black', alpha=0.7)
    ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Breakeven')
    ax.set_xlabel('P&L ($)', fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)
    ax.set_title('Trade P&L Distribution', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No trades were generated.")

## Next Steps

Now that you've completed the quick start:

1. **Explore Examples:** Run the example scripts in `/examples`
2. **Read Documentation:** See `/docs/USER_GUIDE.md` for detailed usage
3. **Create Custom Strategy:** Use the template in `example_04_custom_strategy.py`
4. **Use Real Data:** Connect to a Dolt database with actual market data
5. **Generate Reports:** Use `Dashboard` and `ReportGenerator` for comprehensive analysis

Happy backtesting!